# Loading data

In [125]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import time 

jobs = pd.read_csv('./csv_out/stackoverflow_jobs_enhanced.csv', thousands=',')

# Filtering for important columns & splitting 

In [127]:
tmp = jobs.loc[:,['jobid', 'city', 'state', 'country', 'tags', 'weeknum', 'salary', 'salary_low', 'salary_high', 'currency', 'equity']]

tag_split = lambda x: pd.Series([i for i in x[1:-1].split(',')])
tag_splitted = tmp['tags'].apply(tag_split)

tag_splitted = tag_splitted.fillna('')

In [128]:
t = []

for i in range(0, len(tmp)):    
    num_tags = len(tag_splitted.iloc[i])
    for j in range(0,num_tags):
        tech = tag_splitted.iloc[i][j]        
        if tech == '': break
        # creating new row 
        new_row = {}
        new_row['jobid'] = tmp.iloc[i]['jobid']
        new_row['city'] = tmp.iloc[i]['city']
        new_row['state'] = tmp.iloc[i]['state']
        new_row['country'] = tmp.iloc[i]['country']
        new_row['weeknum'] = tmp.iloc[i]['weeknum']
        new_row['salary_low'] = tmp.iloc[i]['salary_low']
        new_row['salary_high'] = tmp.iloc[i]['salary_high']
        new_row['equity'] = tmp.iloc[i]['equity']
        new_row['currency'] = tmp.iloc[i]['currency']            
        new_row['tech'] = tech
        # adding 
        t.append(new_row)
        
technologies = pd.DataFrame(t)        

In [129]:
technologies.dtypes

city           object
country        object
currency       object
equity           bool
jobid           int64
salary_high     int64
salary_low      int64
state          object
tech           object
weeknum         int64
dtype: object

In [141]:
# getting the mean figure for salary
technologies['salary_mean'] = technologies[['salary_high','salary_low']].mean(axis=1)

technologies.fillna('', inplace=True)
technologies.head(355)

,city,country,currency,equity,jobid,salary_high,salary_low,state,tech,weeknum,salary_mean
0,Saint Petersburg,US,$,True,125541,75000,45000,FL,angularjs,43,60000
1,Saint Petersburg,US,$,True,125541,75000,45000,FL,html5,43,60000
2,Saint Petersburg,US,$,True,125541,75000,45000,FL,css3,43,60000
3,Saint Petersburg,US,$,True,125541,75000,45000,FL,javascript,43,60000
4,Saint Petersburg,US,$,True,125541,75000,45000,FL,rest,43,60000
5,East Greenwich,US,,False,125540,0,0,RI,c++,43,0
6,East Greenwich,US,,False,125540,0,0,RI,boost,43,0
7,East Greenwich,US,,False,125540,0,0,RI,stl,43,0
8,East Greenwich,US,,False,125539,0,0,RI,c++,43,0
9,East Greenwich,US,,False,125539,0,0,RI,python,43,0


In [ ]:
# Number of techs by city 

In [162]:
technologies[technologies.city == 'London'].groupby(['city', 'tech'])['jobid'].count().sort_values()

city    tech                   
London  "hapi"                       1
         voldemort                   1
         vmware                      1
         visual-studio-2012          1
         visual-studio               1
         vidispine                   1
         video                       1
         vert.x                      1
         vast                        1
         ux                          1
         uwp-xaml                    1
         uwp                         1
         user-interface              1
         user-experience             1
        "systems"                    1
         unityscript                 1
        "t-sql"                      1
         umbraco                     1
         ui                          1
        "tableau"                    1
         turbogears2                 1
         tornado                     1
        "task-parallel-library"      1
        "tcserver"                   1
         vtk                    

In [150]:
technologies[technologies.city == 'London'].sort_values(by='tech')

,city,country,currency,equity,jobid,salary_high,salary_low,state,tech,weeknum,salary_mean
12446,London,UK,,False,124055,0,0,,.net,35,0.0
24032,London,UK,£,True,107332,75000,35000,,.net,42,55000.0
1263,London,UK,,False,121818,0,0,,.net,30,0.0
1045,London,UK,£,False,121844,47303,40000,,.net,30,43651.5
23363,London,UK,£,False,123251,35000,25000,,.net,42,30000.0
16011,London,UK,,False,120496,0,0,,.net,38,0.0
20455,London,UK,£,False,122852,55000,45000,,.net,41,50000.0
3818,London,UK,,False,89913,0,0,,.net,31,0.0
23109,London,UK,,False,127017,0,0,,.net,42,0.0
23798,London,UK,£,False,127100,60000,40000,,.net,42,50000.0


In [163]:
tag_splitted

,0,1,2,3,4,5
0,angularjs,html5,css3,javascript,rest,
1,c++,boost,stl,,,
2,c++,python,,,,
3,python,amazon-web-services,linux,,,
4,c++,embedded,reverse-engineering,java,python,
5,user-interface,python,chemistry,physics,oop,
6,security,encryption,cryptography,amazon-web-services,authentication,
7,scikit-learn,apache-spark,pyspark,numpy,,
8,c++,linux-kernel,dpdk,ip,networking,
9,go,postgresql,distributed-computing,amazon-web-services,,


In [165]:
tmp.tags

0               [angularjs, html5, css3, javascript, rest]
1                                        [c++, boost, stl]
2                                            [c++, python]
3                     [python, amazon-web-services, linux]
4        [c++, embedded, reverse-engineering, java, pyt...
5        [user-interface, python, chemistry, physics, oop]
6        [security, encryption, cryptography, amazon-we...
7             [scikit-learn, apache-spark, pyspark, numpy]
8                [c++, linux-kernel, dpdk, ip, networking]
9        [go, postgresql, distributed-computing, amazon...
10       [python, django, javascript, jenkins, continuo...
11                   [c#, .net, asp.net, html, javascript]
12        [html5, css, java, angularjs, twitter-bootstrap]
13                       [python, django, react.js, mysql]
14                [c#, web-services, asp.net-mvc, xamarin]
15       [selenium, phantomjs, automated-tests, qa, pyt...
16       [jira, confluence, produktmanager, spezifikati.